# Importing MIMIC-III v1.4 data into a graph database

### About the data  
The data were obtaind from multiple repositories on Physionet. The main body of data comes from the MIMIC-III v1.4 Clinical Database. 

#### MIMIC-III Clinical Database  
Description:
MIMIC-III is a large, freely-available database comprising deidentified health-related data associated with over forty thousand patients who stayed in critical care units of the Beth Israel Deaconess Medical Center between 2001 and 2012. The database includes information such as demographics, vital sign measurements made at the bedside (~1 data point per hour), laboratory test results, procedures, medications, caregiver notes, imaging reports, and mortality (including post-hospital discharge).   
Published: Sept. 4, 2016. Version: 1.4  
Data downloaded 13 February 2021 at https://physionet.org/content/mimiciii/1.4/

#### Phenotype Annotations for Patient Notes in the MIMIC-III Database  
Description:  
"A dataset of patient notes, all in the English language, with a focus on frequently readmitted patients, labeled with 15 clinical patient phenotypes believed to be associated with risk of recurrent Intensive Care Unit (ICU) readmission per our domain experts (co-authors LAC, PAT, DAG) as well as the literature [5-7]. Each entry in this database consists of a MIMIC-III derived Subject Identifier ("SUBJECT_ID", integer), a Hospital Admission Identifier ("HADM_ID", integer), the index from MIMIC-III v1.4 NOTEEVENTS table ("ROW_ID", integer), 15 Phenotypes (binary) including "None'' and "Unsure'', and Operator (string)."  
Published: March 5, 2020. Version: 1.20.03  
Data downloaded 13 February 2021 at https://physionet.org/content/phenotype-annotations-mimic/1.20.03/

## Import Data into a graph database
---

The downloaded data consisted of the following 27 CSV files, which collectively used 46.6 GB of memory:
- ADMISSIONS.csv
- CALLOUT.csv
- CAREGIVERS.csv
- CHARTEVENTS.csv
- CPTEVENTS.csv
- DATETIMEEVENTS.csv
- D_CPT.csv
- DIAGNOSES_ICD.csv
- D_ICD_DIAGNOSES.csv
- D_ICD_PROCEDURES.csv
- D_ITEMS.csv
- D_LABITEMS.csv
- DRGCODES.csv
- ICUSTAYS.csv
- INPUTEVENTS_CV.csv
- INPUTEVENTS_MV.csv
- LABEVENTS.csv
- MICROBIOLOGYEVENTS.csv
- NOTEEVENTS_ANNOTATION.csv
- NOTEEVENTS.csv
- OUTPUTEVENTS.csv
- PATIENTS.csv
- PRESCRIPTIONS.csv
- PROCEDUREEVENTS_MV.csv
- PROCEDURES_ICD.csv
- SERVICES.csv
- TRANSFERS.csv

These files were placed in the Import folder of the MIMIC-III Neo4j database to make them readily available for import into the graph. The name of the file containing phenotype annotations was changed from ACTdb102003.csv to NOTEEVENTS_ANNOTATIONS.csv to improve readability.

In [1]:
import pandas as pd
from progressbar import ProgressBar
pd.set_option('display.max_colwidth', None)
import time
import re
import pprint
import numpy as np
import multiprocessing
from multiprocessing import  Pool

In [2]:
# import getpass
# password = getpass.getpass("\nPlease enter the Neo4j database password to continue \n")
password = 'NikeshIsCool'

from neo4j import GraphDatabase
driver=GraphDatabase.driver(uri="bolt://localhost:7687", auth=('neo4j',password))
session=driver.session()

In [2]:
# Specify the location of the CSV files to import. This is the path to the /import folder in your Neo4j database. 
# Make sure your path ends with a backslash
path = '/home/tim/.config/Neo4j Desktop/Application/relate-data/dbmss/dbms-8b11ca35-a988-4add-ac4e-3397432fb55a/import/'

# Create a list of all CSV files to import
csv_files = ['ADMISSIONS.csv', 'CALLOUT.csv', 'CAREGIVERS.csv', 'CHARTEVENTS.csv', 'CPTEVENTS.csv', 'DATETIMEEVENTS.csv', 'D_CPT.csv', 'DIAGNOSES_ICD.csv', 'D_ICD_DIAGNOSES.csv', 'D_ICD_PROCEDURES.csv', 'D_ITEMS.csv', 'D_LABITEMS.csv', 'DRGCODES.csv', 'ICUSTAYS.csv', 'INPUTEVENTS_CV.csv', 'INPUTEVENTS_MV.csv', 'LABEVENTS.csv', 'MICROBIOLOGYEVENTS.csv', 'NOTEEVENTS_ANNOTATION.csv', 'NOTEEVENTS.csv', 'OUTPUTEVENTS.csv', 'PATIENTS.csv', 'PRESCRIPTIONS.csv', 'PROCEDUREEVENTS_MV.csv', 'PROCEDURES_ICD.csv', 'SERVICES.csv', 'TRANSFERS.csv']

# Create a dictionary with file names as keys and the list of headers for each file as values 
headers_dict = {}
for file in csv_files:
    headers = pd.read_csv(path+file, nrows=1)
    headers = headers.columns.tolist()
    if not file in headers_dict:
        headers_dict[file] = headers
        
# Inspect an example item in the dictionary
print(headers_dict['ADMISSIONS.csv'])

['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS', 'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA']


In [3]:
# Create a function that writes the string for a cypher command
# to create nodes from each CSV file

def csv_to_node(csv_file):
    
    # Create the node label based on the CSV file name. Place it in title case and remove the '.csv' suffix
    label= csv_file[:-4].title() 
    
    # Convert the CSV's headers into node properties
    properties = '{'
    col_index = 0
    for header in headers_dict[csv_file]:
        properties = properties+header+':column['+str(col_index)+'], '
        properties = properties.replace('.','_')
        properties = properties.lower()
        col_index += 1
    properties = properties[:-2]+'}' # Delete last comma of the list and add the ending curly bracket
    
    # Compile the complete cypher command
    cypher = '''USING PERIODIC COMMIT 100000 LOAD CSV WITH HEADERS FROM "file:///{csv_file}" AS column CREATE (n:{label} {properties})'''.format(csv_file=csv_file, label=label, properties=properties)
    return cypher

# Generate the cypher code for a single csv file to test in the Neo4j browser
csv_to_node('NOTEEVENTS_ANNOTATION.csv')

'USING PERIODIC COMMIT 100000 LOAD CSV FROM "file:///NOTEEVENTS_ANNOTATION.csv" AS column CREATE (n:Noteevents_Annotation {hadm_id:column[0], subject_id:column[1], row_id:column[2], advanced_cancer:column[3], advanced_heart_disease:column[4], advanced_lung_disease:column[5], alcohol_abuse:column[6], batch_id:column[7], chronic_neurological_dystrophies:column[8], chronic_pain_fibromyalgia:column[9], dementia:column[10], depression:column[11], developmental_delay_retardation:column[12], non_adherence:column[13], none:column[14], obesity:column[15], operator:column[16], other_substance_abuse:column[17], schizophrenia_and_other_psychiatric_disorders:column[18], unsure:column[19]})'

### Create all nodes

In [6]:
# Create all nodes
for csv_name in csv_files:
    query = csv_to_node(csv_name)
    session.run(query)

This operation created 397,846,346 nodes.

### Re-format all datetime stamps to be recognized as datetime by Neo4j

In [8]:
# Identify all the time fields in MIMIC-III. Note that chartdate was omitted because it's redundant
# and it's formatted differently than all the other datetimes
time_fields = pd.read_csv('MIMIC-III_timestamp_fields.csv')
time_fields.loc[:,'Table'] = time_fields.loc[:,'Table'].str.title()
time_fields.loc[:,'Column'] = time_fields.loc[:,'Column'].str.lower()
time_fields.head()

,Table,Column
0,Admissions,admittime
1,Admissions,deathtime
2,Admissions,dischtime
3,Admissions,edouttime
4,Admissions,edregtime


In [10]:
# Set all the datetime strings as Neo4j temporal instant types
for index, row in time_fields.iterrows():
    Label = row[0]
    prop = row[1]
    query = '''
    'MATCH (n:{Label})
    WHERE EXISTS(n.{prop})
    RETURN n',
    'SET n.{prop} = datetime(REPLACE(n.{prop}, " ", "T"))'
    '''.format(Label=Label, prop=prop)
    
    command = 'CALL apoc.periodic.iterate('+query+', {batchSize:1000, parallel: true, iterateList:true})'
    session.run(command)

### Create relationships

Prepare a CSV of the tables and foreign keys from the original MIMIC III schema:
- Obtain foreign key constraints from https://mit-lcp.github.io/mimic-schema-spy/constraints.html
- Copy the table into a CSV file. 
- In a spreadsheet editor, keep only the columns "Child Column" and "Parent Column." 
- Split each of these columns on "." into node and foreign key columns. 
- Add a "RELATIONSHIP_NAME" column and create a short but descriptive relationship name for each parent-child pair
- Save as "mimic3_MI1_relational_schema.csv"

In [4]:
# Read the CSV into a dataframe
sql_schema = pd.read_csv('mimic3_MI1_relational_schema.csv')

# Examine the first five rows
sql_schema.iloc[:5,:]

,Child Node,Child Foreign Key,Parent Node,Parent Foreign Key,RELATIONSHIP_NAME
0,callout,hadm_id,admissions,hadm_id,OCCURRED_DURING
1,caregivers,cgid,chartevents,cgid,CHARTED
2,caregivers,cgid,datetimeevents,cgid,CHARTED
3,caregivers,cgid,inputevents_cv,cgid,CHARTED
4,caregivers,cgid,inputevents_mv,cgid,CHARTED


Note that the normal cypher command to create these relationships would attempt to load too much into RAM at the same time, so the computer can't run the command unless you utilize periodic execution.  
See the Neo4j documentation for periodic execution at https://neo4j.com/labs/apoc/4.1/graph-updates/periodic-execution/ to understand the cypher command in the following cell.  

Note also that we avoid creating a cartesian product with our MATCH query, which would be very computationally expensive. See Stefan Armbruster's description of how to avoid creating a cartesian product in this scenario at https://community.neo4j.com/t/reliably-create-relationships-on-12million-nodes/22223.

In [5]:
# Write a cypher command for each relationship specified in the original
# MIMIC III schema
count = 0
for index, row in sql_schema.iterrows():
    child_node = row['Child Node'].title()
    child_fk = row['Child Foreign Key'].lower()
    parent_node = row['Parent Node'].title()
    parent_fk = row['Parent Foreign Key'].lower()
    RELATIONSHIP_NAME = row['RELATIONSHIP_NAME']

    command = 'CALL apoc.periodic.iterate(\"MATCH (cn:{child_node}) MATCH (pn:{parent_node} {{{parent_fk}:cn.{child_fk}}}) RETURN cn, pn\", \"CREATE (cn)-[:{RELATIONSHIP_NAME}]->(pn)\", {{batchSize:10000, parallel: true, iterateList:true}})'.format(child_node=child_node, parent_node=parent_node, child_fk=child_fk, parent_fk=parent_fk, RELATIONSHIP_NAME=RELATIONSHIP_NAME)
    session.run(command)
    count += 1
    print(str(count)+' of 63: '+child_node)

1 of 63: Callout
2 of 63: Caregivers
3 of 63: Caregivers
4 of 63: Caregivers
5 of 63: Caregivers
6 of 63: Caregivers
7 of 63: Caregivers
8 of 63: Caregivers
9 of 63: Chartevents
10 of 63: Chartevents
11 of 63: Cptevents
12 of 63: D_Icd_Diagnoses
13 of 63: D_Icd_Procedures
14 of 63: D_Items
15 of 63: D_Items
16 of 63: D_Items
17 of 63: D_Items
18 of 63: D_Items
19 of 63: D_Items
20 of 63: D_Items
21 of 63: D_Items
22 of 63: D_Labitems
23 of 63: Datetimeevents
24 of 63: Datetimeevents
25 of 63: Diagnoses_Icd
26 of 63: Drgcodes
27 of 63: Icustays
28 of 63: Inputevents_Cv
29 of 63: Inputevents_Cv
30 of 63: Inputevents_Mv
31 of 63: Inputevents_Mv
32 of 63: Labevents
33 of 63: Microbiologyevents
34 of 63: Noteevents
35 of 63: Outputevents
36 of 63: Outputevents
37 of 63: Patients
38 of 63: Patients
39 of 63: Patients
40 of 63: Patients
41 of 63: Patients
42 of 63: Patients
43 of 63: Patients
44 of 63: Patients
45 of 63: Patients
46 of 63: Patients
47 of 63: Patients
48 of 63: Patients
49 of 

The above code created 1,881,913,850 relationships

In [6]:
# Create relationshipS for Noteevents_Annotation table
nea_schema = [
    ['Child Node', 'Child Foreign Key', 'Parent Node', 'Parent Foreign Key', 'RELATIONSHIP_NAME'],
    ['Patients', 'subject_id', 'Noteevents_Annotation', 'subject_id', 'HAD'],
    ['Noteevents_Annotation', 'hadm_id', 'Admissions', 'hadm_id', 'OCCURRED_DURING'],
    ['Noteevents_Annotation', 'row_id', 'Noteevents', 'row_id', 'BASED_ON']
]

count = 0
for row in nea_schema[1:]:
    child_node = row[0].title()
    child_fk = row[1].lower()
    parent_node = row[2].title()
    parent_fk = row[3].lower()
    RELATIONSHIP_NAME = row[4]

    command = 'CALL apoc.periodic.iterate(\"MATCH (cn:{child_node}) MATCH (pn:{parent_node} {{{parent_fk}:cn.{child_fk}}}) RETURN cn, pn\", \"CREATE (cn)-[:{RELATIONSHIP_NAME}]->(pn)\", {{batchSize:10000, parallel: true, iterateList:true}})'.format(child_node=child_node, parent_node=parent_node, child_fk=child_fk, parent_fk=parent_fk, RELATIONSHIP_NAME=RELATIONSHIP_NAME)
    session.run(command)
    count += 1
    print(str(count)+' of 3: '+child_node)

1 of 3: Patients
2 of 3: Noteevents_Annotation
3 of 3: Noteevents_Annotation


The relationship-building commands above operated on 397,848,371 nodes, creating a total of 1,881,920,681 relationships. These relationships used a total of 149GB of disk space.

<a id='UMLS_problem_creation'></a>
### Create problem nodes

To-do:
- Add CUIs to all problems

In [4]:
def get_notes(query):

    # Execute the cypher query to obtain physician notes in a Neo4j object
    data = session.run(query)

    # Convert the neo4j object into a dataframe
    df = pd.DataFrame([dict(record) for record in data])

    # Define a function that takes a full physician's note and returns only the Assessment and Plan portion
    def isolate_AP(note):
        # Define a regex pattern to isolate the Assessment and Plan portion of each note
        try:
            pattern = re.compile(r'(Assessment and Plan.*)ICU Care', re.DOTALL)
            return re.search(pattern, note).group(1)
        except:
            pattern = re.compile(r'(Assessment and Plan.*)ICU \[\*\*', re.DOTALL)
            if re.search(pattern, note) != None:
                return re.search(pattern, note).group(1)
            else:
                return 'No AP isolated'

    # Apply the function that isolates AP to the column of the dataframe containing notes
    df['note'] = df['note'].apply(isolate_AP)

    # Check the output
    return df

In [5]:
# Cypher query to obtain all physician notes for a patient with a given subject_id
query = '''
MATCH (n:Noteevents)
WHERE n.category = "Physician "
RETURN DISTINCT(n.text) AS note, n.storetime as storetime, n.subject_id as subject_id, n.row_id as note_id
ORDER BY subject_id, storetime'''

start_time = time.time()
df = get_notes(query)
print("Total runtime:", time.time() - start_time, "seconds")
df.info()

Total runtime: 17.153480768203735 seconds
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141624 entries, 0 to 141623
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   note        141624 non-null  object
 1   storetime   141624 non-null  object
 2   subject_id  141624 non-null  object
 3   note_id     141624 non-null  object
dtypes: object(4)
memory usage: 4.3+ MB


In [21]:
query = '''
MATCH (n:Noteevents)
WHERE n.category = "Physician "
RETURN n.storetime as storetime, n.subject_id as subject_id, n.row_id as note_id
ORDER BY subject_id, storetime
LIMIT 5'''

# Execute the cypher query to obtain physician notes in a Neo4j object
data = session.run(query)

# Convert the neo4j object into a dataframe
df = pd.DataFrame([dict(record) for record in data])

,storetime,subject_id,note_id
0,2142-06-02T10:22:01.000000000+00:00,10134,377333
1,2142-06-02T12:13:26.000000000+00:00,10134,377338
2,2142-06-03T06:53:24.000000000+00:00,10134,377425
3,2142-06-03T06:57:49.000000000+00:00,10134,377428
4,2142-06-03T10:35:24.000000000+00:00,10134,377448


In [23]:
type(df.iloc[0].storetime)

neo4j.time.DateTime

In [6]:
str_to_aui = pd.read_csv('MRCONSO_for_import.csv', usecols=['AUI', 'STR', 'STY'], encoding='utf-8')
str_to_aui.head()

,AUI,STR,STY
0,A26634265,(131)I-Macroaggregated Albumin,"Amino Acid, Peptide, or Protein"
1,A26634265,(131)I-Macroaggregated Albumin,Pharmacologic Substance
2,A26634265,(131)I-Macroaggregated Albumin,"Indicator, Reagent, or Diagnostic Aid"
3,A26634266,(131)I-MAA,"Amino Acid, Peptide, or Protein"
4,A26634266,(131)I-MAA,Pharmacologic Substance


In [7]:
str_to_aui.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11679703 entries, 0 to 11679702
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   AUI     object
 1   STR     object
 2   STY     object
dtypes: object(3)
memory usage: 267.3+ MB


In [8]:
include_types = ['Disease or Syndrome', 'Pathologic Function', 'Neoplastic Process', 'Sign or Symptom', 'Injury or Poisoning', 'Mental or Behavioral Dysfunction']
str_to_aui = str_to_aui[str_to_aui.STY.isin(include_types)]

In [9]:
str_to_aui.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338294 entries, 4491 to 11679528
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   AUI     1338294 non-null  object
 1   STR     1338294 non-null  object
 2   STY     1338294 non-null  object
dtypes: object(3)
memory usage: 40.8+ MB


In [10]:
str_to_aui.sort_values(by='STR', inplace=True)
str_to_aui

,AUI,STR,STY
5173269,A23028193,#Ankle,Injury or Poisoning
5173270,A22851845,#Ankle,Injury or Poisoning
5173274,A22973018,#Ankle (& [tibia]),Injury or Poisoning
5173273,A4983548,#Ankle (& [tibia]),Injury or Poisoning
5173272,A4983547,#Ankle (& [tibia]),Injury or Poisoning
...,...,...,...
2015936,A31182206,{DRINK},Mental or Behavioral Dysfunction
1454962,A3806061,État marbré,Disease or Syndrome
5362742,A31510015,α-gal allergy,Pathologic Function
11410947,A31501148,α-gal sensitization,Pathologic Function


In [11]:
# Define a function that conducts a fast binary search on a sorted column of a dataframe, returning only full match results.

def binary_search(dataframe, column, target):
    range_start = 0
    range_end = len(dataframe)-1
    while range_start < range_end:
        range_middle = (range_end + range_start) // 2
        value = dataframe.iloc[range_middle][column]
        if value == target:
            return dataframe.iloc[range_middle]
        elif value < target:
            # Discard the first half of the range
            range_start = range_middle + 1
        else:
            # Discard the second half of the range
            range_end = range_middle - 1
    # At this point range_start = range_end
    value = dataframe.iloc[range_start][column]
#     return value
    if value == target:
        return dataframe.iloc[range_start]
    else:
        return 0

In [12]:
# Define a function which takes a string and returns the problem identified at the start of the string

def text_to_AUIs(text):
    
    # Remove any non-alphanumeric characters, set the encodning to unicode, and split the text into a list of words
    text = re.sub('[\W_]+', ' ', text, flags=re.UNICODE)
    text = text.strip()
    text = text.split(' ')
    
    aui = None
    
    # Iterate through the beginning of the list of words to find the largest set of consecutive words that match aui-associated strings
    
    for i in reversed(range(1,7)):
        term = ' '.join(text[:i])
        frame = binary_search(dataframe = str_to_aui, column = 'STR', target = term)
        
        if type(frame) == int:
            frame = binary_search(dataframe = str_to_aui, column = 'STR', target = term.lower())
            if type(frame) == int:
                frame = binary_search(dataframe = str_to_aui, column = 'STR', target = term.upper())
                if type(frame) == int:
                    frame = binary_search(dataframe = str_to_aui, column = 'STR', target = term.title())
                    if type(frame) == int:
                        pass
                      
                    else:
                        return frame['AUI']
                else:
                    return frame['AUI']
            else:
                return frame['AUI']
        else:
            return frame['AUI']
        
            break

In [52]:
# Define a function that accepts an Assessment and Plan and extracts the problems listed
def AP_to_problems(AP):
    problem_set = set([])
    if AP != 'No AP isolated':
        rows_list = AP.split('\n')
        for row in rows_list:
            aui = text_to_AUIs(row)
            if aui != None:
                problem_set.add(aui)
        return problem_set
    else:
        return problem_set
        
# Define a function that applies the AP_to_problems function to the note column of a 
# dataframe and adds the result as a new "problem" column to the dataframe
def add_problems_col(AP_series):
    AUIs_series = AP_series.apply(AP_to_problems)
    return AUIs_series

# Define a function that uses parallel processing to perform a given function 
# on a given dataframe
def parallelize_dataframe(AP_series, func):
#     n_cores = multiprocessing.cpu_count()
    n_cores = 11
    series_split = np.array_split(AP_series, n_cores)
    pool = Pool(n_cores)
    AUIs_series = pd.concat(pool.map(func, series_split))
    pool.close()
    pool.join()
    return AUIs_series

In [54]:
start_time = time.time()

AP_series = df['note']

# Run the function that adds a "problems" column to the dataframe, using the 
# parallel processing function
AUI_series = parallelize_dataframe(AP_series, add_problems_col)
df['aui'] = AUI_series

print("Total runtime:", time.time() - start_time, "seconds")

Total runtime: 38891.607382535934 seconds


In [63]:
# Keep only the rows that have Assessment and Plan isolated
df = df[df['note'] != 'No AP isolated']
df = df[['storetime', 'subject_id', 'note_id', 'aui']].explode(column='aui')
df.dropna(inplace=True)
df.head()

,storetime,subject_id,note_id,aui
0,2142-06-02T10:22:01.000000000+00:00,10134,377333,A27377006
0,2142-06-02T10:22:01.000000000+00:00,10134,377333,A23970434
0,2142-06-02T10:22:01.000000000+00:00,10134,377333,A31604611
0,2142-06-02T10:22:01.000000000+00:00,10134,377333,A1307145
0,2142-06-02T10:22:01.000000000+00:00,10134,377333,A0482736


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1033880 entries, 0 to 141623
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   storetime   1033880 non-null  object
 1   subject_id  1033880 non-null  object
 2   note_id     1033880 non-null  object
 3   aui         1033880 non-null  object
dtypes: object(4)
memory usage: 39.4+ MB


In [65]:
# Write out to CSV
df.to_csv('problem_AUIs_from_notes_3Nov2021.csv', index=False, encoding='utf-8')

Move the CSV file to the import file

In [4]:
# Create a constraint and index the Patients subject_id property
command = '''
CREATE CONSTRAINT Patient_subject_id IF NOT EXISTS
ON (n:Patients)
ASSERT n.subject_id IS UNIQUE
'''
session.run(command)

In [5]:
# Create an index on the Concepts aui property
command = '''
CREATE INDEX Concept_AUI IF NOT EXISTS FOR (c:Concept) ON (c.aui) 
'''
session.run(command)

In [24]:
# Batch import the CSV into the database

start_time = time.time()

command = '''
USING PERIODIC COMMIT 1000 LOAD CSV WITH HEADERS FROM "file:///problem_AUIs_from_notes_3Nov2021.csv" AS csvLine
MATCH (pt:Patients {subject_id:csvLine.subject_id})
CREATE (prob:Problem {aui:csvLine.aui, datetime_reported:datetime(csvLine.storetime)})
CREATE (pt)-[:HAD_PROBLEM]->(prob)
WITH prob
MATCH (c:Concept)
WHERE prob.aui = c.aui
SET prob.description = c.term
'''
session.run(command)

print("Total runtime:", time.time() - start_time, "seconds")

Total runtime: 32.99302959442139 seconds


In [25]:
# Let's do a sanity check on the 400 most common problems in the database:
command = '''MATCH (p:Problem)
RETURN p.description, p.aui, count(*) AS number 
ORDER BY number DESC
LIMIT 400'''
data = session.run(command)

pd.set_option("display.max_rows", 400)

top_problems = pd.DataFrame([dict(record) for record in data])
print(top_problems)

                             p.description      p.aui  number
0                                    Renal  A14225102   37759
1                                   Wounds  A26663781   25728
2                       Infectious Disease   A0073713   25615
3                      respiratory failure   A1192377   18838
4                                   Anemia  A24094711   16117
5                                     Pain  A29591172   11500
6                                      HTN   A1307145   10200
7                                       ID  A27371099    9822
8                                      DVT  A21144946    9709
9                                     PLAN  A16611948    9002
10                   altered mental status  A18655171    8927
11                                     IVF  A28801973    8925
12                                  sepsis  A23895141    8881
13                                     CAD  A23970434    8263
14                     Respiratory failure  A28745064    8031
15      

In [26]:
# Note all the problems which appear to be artifacts of the method of extraction
top_problems.iloc[[0, 7, 9, 19, 21, 24, 26, 28, 33, 35, 37, 40, 42, 43, 47, 57, 59, 60, 65, 76, 86, 89, 93, 95, 101, 107, 115, 128, 135, 141, 147, 150, 151, 165, 176, 177, 185, 207, 213, 218, 220, 223, 229, 231, 235, 238, 265, 298, 301, 332, 348, 369, 374, 388, 391]]

,p.description,p.aui,number
0,Renal,A14225102,37759
7,ID,A27371099,9822
9,PLAN,A16611948,9002
19,IS,A12006040,6994
21,AS,A12021952,6844
24,Blood,A14224321,6717
26,PER,A11932691,5855
28,but,A18579364,5713
33,WAS,A12030424,5165
35,disease,A18683747,4932


In [27]:
# Create a list of AUIs for deletion
delete_list = top_problems['p.aui'].to_list()

# Delete the extraneous "problems" in the delete_list
command = '''
MATCH (p:Problem)
WHERE p.aui IN {delete_list}
DETACH DELETE p'''.format(delete_list=delete_list)
session.run(command)

In [6]:
# Create an index on the Problem aui property
command = '''
CREATE INDEX Problem_AUI IF NOT EXISTS FOR (c:Problem) ON (c.aui) 
'''
session.run(command)

In [9]:
# Connect problem nodes with an INSTANCE_OF relationship to Concept nodes

start_time = time.time()

query = '''
"MATCH (p:Problem)
MATCH (c:Concept)
WHERE p.aui = c.aui
RETURN p, c",
"CREATE (p)-[:INSTANCE_OF]->(c)"'''
command = 'CALL apoc.periodic.iterate('+query+', {batchSize:1000, parallel: true, iterateList:true})'
session.run(command)

print("Total runtime:", time.time() - start_time, "seconds")

Total runtime: 7.896453142166138 seconds


In [13]:
# Add CUIs to all problems
command = '''
MATCH (p:Problem)-[:INSTANCE_OF]->(c:Concept)
SET p.cui = c.cui
'''
session.run(command)

In [30]:
# Finally, let's connect our new problem nodes to the notes from which the problems were extracted
start_time = time.time()

command = '''
MATCH (n:Noteevents)<-[:HAD]-(pt:Patients)-[:HAD_PROBLEM]->(p:Problem)
WHERE p.datetime_reported = n.storetime
MERGE (n)-[:REPORTED]->(p)
'''
session.run(command)

print("Total runtime:", time.time() - start_time, "seconds")

Total runtime: 16.77986478805542 seconds


<a id='MIMIC_Rx_to_UMLS'></a>
### Connect MIMIC-III's Prescription nodes with `INSTANCE_OF` relationships to Concept nodes

In [8]:
# Add AUI as a property to MIMIC-III's medication nodes, then connect them with a "instance_of" relationship
# to the UMLS concept nodes that share the same AUI. See documentation at https://www.nlm.nih.gov/research/umls/rxnorm/docs/techdoc.html
# and https://www.nlm.nih.gov/research/umls/rxnorm/docs/techdoc.html#sat

# Load the RXNSAT.RRF file, which has NDC codes mapped to AUIs
path = '/media/sata_1TB_internal/RxNorm_full_06072021/rrf/'
rxnsat = pd.read_csv(path + 'RXNSAT.RRF', usecols=[3,4,8,10], sep='|', header=None)
rxnsat.columns = ['RXAUI','STYPE','ATN','ATV']
# rxnsat = pd.read_csv(path + 'RXNSAT.RRF', sep='|', header=None)
# rxnsat.columns = ['RXCUI','LUI','SUI','RXAUI','STYPE','CODE','ATUI','SATUI','ATN','SAB','ATV','SUPPRESS','CVF','']
rxnsat.head()

,RXAUI,STYPE,ATN,ATV
0,829,AUI,RXN_BN_CARDINALITY,single
1,8056626,AUI,TERMUI,T005606
2,8056626,AUI,LT,TRD
3,8056626,AUI,TH,UNK (19XX)
4,8056626,SCUI,RN,0


In [9]:
rxnsat = rxnsat[(rxnsat.ATN == 'NDC') & (rxnsat.STYPE == 'AUI')]
rxnsat = rxnsat[['RXAUI', 'ATV']]
rxnsat.columns = ['SAUI','NDC']
rxnsat

,SAUI,NDC
60045,1473211,000395021301
60046,1473211,000395021391
60047,1473211,017317004101
60048,1473211,017317004105
60049,1473211,049452815001
...,...,...
6938451,12639965,13985-433-05
6938453,12640445,11695697701
6938454,12640445,13985043305
6938455,12640445,46066006115


In [10]:
path = '/media/sata_1TB_internal/umls-2020AB-full/2020AB/META/'
mrconso = pd.read_csv(path + 'MRCONSO.RRF', sep='|', usecols=[7,8,11,14], header=None, encoding='utf-8')
mrconso.columns = ['AUI','SAUI', 'SAB', 'STR']
mrconso.head()

,AUI,SAUI,SAB,STR
0,A26634265,NaN,MSH,(131)I-Macroaggregated Albumin
1,A26634266,NaN,MSH,(131)I-MAA
2,A13433185,NaN,MSHFRE,Macroagrégats d'albumine marquée à l'iode 131
3,A27488794,NaN,MSHFRE,MAA-I 131
4,A27614225,NaN,MSHFRE,Macroagrégats d'albumine humaine marquée à l'iode 131


In [12]:
# Select only rows pertaining to RXNORM
mrconso = mrconso[mrconso.SAB == 'RXNORM']

# Set SAUI object type as int64 
mrconso.dropna(inplace=True)
mrconso.SAUI = mrconso.SAUI.astype('int64')
mrconso

,AUI,SAUI,SAB,STR
6,A28315139,9194921,RXNORM,"1,2-dipalmitoylphosphatidylcholine"
1320,A10336090,829,RXNORM,Parlodel
1489,A31762929,12251526,RXNORM,mesna
2183,A31685600,12254325,RXNORM,droxidopa
2304,A31642632,12254378,RXNORM,beta-alanine
...,...,...,...,...
15228884,A32333738,12426223,RXNORM,Risdiplam Oral Product
15228885,A32337625,12427168,RXNORM,satralizumab Injectable Product
15228886,A32331662,12427754,RXNORM,Oliceridine Injectable Product
15228887,A32331664,12427922,RXNORM,oxybate Oral Liquid Product


In [13]:
# Merge NDC numbers into the mrconso dataframe
mrconso = pd.merge(mrconso, rxnsat, on=['SAUI'])
mrconso.head()

,AUI,SAUI,SAB,STR,NDC
0,A31661492,12330092,RXNORM,calcium chloride 0.0014 MEQ/ML / potassium chloride 0.004 MEQ/ML / sodium chloride 0.103 MEQ/ML / sodium lactate 0.028 MEQ/ML Injectable Solution,00264775000
1,A31661492,12330092,RXNORM,calcium chloride 0.0014 MEQ/ML / potassium chloride 0.004 MEQ/ML / sodium chloride 0.103 MEQ/ML / sodium lactate 0.028 MEQ/ML Injectable Solution,00264775010
2,A31661492,12330092,RXNORM,calcium chloride 0.0014 MEQ/ML / potassium chloride 0.004 MEQ/ML / sodium chloride 0.103 MEQ/ML / sodium lactate 0.028 MEQ/ML Injectable Solution,00264775020
3,A31661492,12330092,RXNORM,calcium chloride 0.0014 MEQ/ML / potassium chloride 0.004 MEQ/ML / sodium chloride 0.103 MEQ/ML / sodium lactate 0.028 MEQ/ML Injectable Solution,00338009202
4,A31661492,12330092,RXNORM,calcium chloride 0.0014 MEQ/ML / potassium chloride 0.004 MEQ/ML / sodium chloride 0.103 MEQ/ML / sodium lactate 0.028 MEQ/ML Injectable Solution,00338011702


In [14]:
mrconso.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253312 entries, 0 to 253311
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   AUI     253312 non-null  object
 1   SAUI    253312 non-null  int64 
 2   SAB     253312 non-null  object
 3   STR     253312 non-null  object
 4   NDC     253312 non-null  object
dtypes: int64(1), object(4)
memory usage: 11.6+ MB


In [15]:
# Get a list of all NDC codes used in MIMIC-III
query = '''
MATCH (rx:Prescriptions)
WHERE size(rx.ndc) = 11
RETURN COLLECT(DISTINCT(rx.ndc)) as NDC_list
'''
data = session.run(query)
NDC_list = data.value()[0]

In [16]:
len(NDC_list)

4203

In [17]:
# Create an index on the Problem aui property
command = '''
CREATE INDEX Prescription_NDC IF NOT EXISTS FOR (c:Prescriptions) ON (c.ndc) 
'''
session.run(command)

In [18]:
pbar = ProgressBar()
for NDC in pbar(NDC_list):
    try:
        AUI = mrconso[mrconso.NDC == NDC].AUI.iloc[0]
        command = '''
        MATCH (instance:Prescriptions {{ndc:'{NDC}'}})
        MATCH (concept:Concept {{aui:'{AUI}'}})
        SET concept.ndc = '{NDC}'
        MERGE (instance)-[:INSTANCE_OF]->(concept)
        '''.format(NDC=NDC, AUI=AUI)
        session.run(command)
    except:
        pass

100% |########################################################################|


<a id='MIMIC_labs_to_UMLS'></a>
### Connect MIMIC-III's Labitems nodes with INSTANCE_OF relationships to Concept nodes

In [3]:
# Load MRCONSO.RRF to get a mapping of LOINC codes to UMLS AUIs
path = '/media/sata_1TB_internal/umls-2020AB-full/2020AB/META/'
mrconso = pd.read_csv(path + 'MRCONSO.RRF', sep='|', usecols=[1,7,9,11], header=None, encoding='utf-8')
# mrconso = pd.read_csv(path + 'MRCONSO.RRF', sep='|', header=None, encoding='utf-8')
# mrconso.columns = ['CUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF', 'AUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE', 'STR', 'SRL', 'SUPPRESS', 'CVF', '']
mrconso.columns = ['LAT','AUI','SCUI', 'SAB']
mrconso.head()

/home/tim/anaconda3/envs/jupyter_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,LAT,AUI,SCUI,SAB
0,ENG,A26634265,M0019694,MSH
1,ENG,A26634266,M0019694,MSH
2,FRE,A13433185,M0019694,MSHFRE
3,FRE,A27488794,M0019694,MSHFRE
4,FRE,A27614225,M0019694,MSHFRE


In [4]:
loinc_codes = mrconso[(mrconso.LAT == 'ENG') & (mrconso.SAB == 'LNC')]
loinc_codes

,LAT,AUI,SCUI,SAB
14,ENG,A18399186,LP15542-1,LNC
16,ENG,A23513030,MTHU010538,LNC
57,ENG,A18243499,LP15278-2,LNC
58,ENG,A23497527,MTHU027241,LNC
505,ENG,A18359748,LP15290-7,LNC
...,...,...,...,...
15193673,ENG,A32072885,LP418799-5,LNC
15193674,ENG,A32074343,LP417224-5,LNC
15193675,ENG,A32075801,LP418432-3,LNC
15193676,ENG,A32075804,LP418814-2,LNC


In [5]:
# Get a list of all LOINC codes used in MIMIC-III
query = '''
MATCH (d:D_Labitems)-[r:DESCRIBES]->(l:Labevents)
RETURN COLLECT(DISTINCT(d.loinc_code)) AS loinc_codes
'''
data = session.run(query)
mimic_loinc_codes = data.value()[0]

In [6]:
len(mimic_loinc_codes)

573

In [7]:
loinc_codes = loinc_codes[loinc_codes.SCUI.isin(mimic_loinc_codes)]
loinc_codes

,LAT,AUI,SCUI,SAB
3233506,ENG,A27139562,702-1,LNC
3233507,ENG,A27130041,702-1,LNC
3233508,ENG,A31410774,702-1,LNC
3233509,ENG,A18357798,702-1,LNC
3233510,ENG,A18357797,702-1,LNC
...,...,...,...,...
11051370,ENG,A18368423,10330-9,LNC
11051371,ENG,A24222560,10330-9,LNC
11051372,ENG,A31418505,10330-9,LNC
11051373,ENG,A18841034,10330-9,LNC


In [8]:
# Create an index on the Labitems loinc_code property
command = '''
CREATE INDEX Labitems_LOINC IF NOT EXISTS FOR (c:D_Labitems) ON (c.loinc_code) 
'''
session.run(command)

In [9]:
for row in loinc_codes.iterrows():
    AUI = row[1].AUI
    LNC = row[1].SCUI
    try:
        command = '''
        MATCH (reference:D_Labitems {{loinc_code:'{LNC}'}})-[r:DESCRIBES]->(instance:Labevents)
        MATCH (concept:Concept {{aui:'{AUI}'}})
        WHERE concept.cui_pref_term IS NOT NULL
        SET concept.loinc_code = '{LNC}', concept.description = reference.label+' in '+reference.fluid
        SET reference.aui = '{AUI}'
        MERGE (instance)-[:INSTANCE_OF]->(concept)
        '''.format(LNC=LNC, AUI=AUI)
        session.run(command)
#         print(command)
    except:
        pass

### Close the connection to the neo4j database

In [21]:
session.close()

### Data references:
The dataset:
Johnson, A., Pollard, T., & Mark, R. (2016). MIMIC-III Clinical Database (version 1.4). PhysioNet. https://doi.org/10.13026/C2XW26.

The original publication:
Johnson, A. E. W., Pollard, T. J., Shen, L., Lehman, L. H., Feng, M., Ghassemi, M., Moody, B., Szolovits, P., Celi, L. A., & Mark, R. G. (2016). MIMIC-III, a freely accessible critical care database. Scientific Data, 3, 160035.

The data hosting service PhysioNet:
Goldberger, A., Amaral, L., Glass, L., Hausdorff, J., Ivanov, P. C., Mark, R., ... & Stanley, H. E. (2000). PhysioBank, PhysioToolkit, and PhysioNet: Components of a new research resource for complex physiologic signals. Circulation [Online]. 101 (23), pp. e215–e220.